In [ ]:
# load datset




import os
import tarfile
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def extract_and_verify_dataset(tar_path="/content/IIIT5K-Word_V3.0.tar"):
    """Extract the IIIT5K dataset and verify its structure"""
    # Create output directory
    output_dir = "iiit5k_dataset"
    if os.path.exists(output_dir):
        print(f"Removing existing directory: {output_dir}")
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    # Extract the tar file
    print(f"Extracting {tar_path} to {output_dir}...")
    try:
        with tarfile.open(tar_path, "r") as tar:
            tar.extractall(output_dir)
        print("Extraction completed successfully!")
    except Exception as e:
        print(f"Error extracting file: {e}")
        return None

    # Function to print directory structure
    def print_directory_structure(path, prefix=""):
        if not os.path.exists(path):
            return

        items = os.listdir(path)
        items.sort()

        for i, item in enumerate(items):
            is_last = i == len(items) - 1
            item_path = os.path.join(path, item)

            # Print item
            print(f"{prefix}{'└── ' if is_last else '├── '}{item}")

            # If it's a directory, recurse
            if os.path.isdir(item_path):
                new_prefix = prefix + ("    " if is_last else "│   ")
                print_directory_structure(item_path, new_prefix)

    # Print the directory structure
    print("\nDataset directory structure:")
    print("="*50)
    print_directory_structure(output_dir)
    print("="*50)

    # Verify critical components
    print("\nVerifying critical dataset components:")

    # Look for train/test directories
    train_dir = None
    test_dir = None
    gt_file = None

    # Search for train directory (could be named various ways)
    for root, dirs, files in os.walk(output_dir):
        for dir_name in dirs:
            if "train" in dir_name.lower() and not any(x in dir_name.lower() for x in ["test", "val"]):
                train_dir = os.path.join(root, dir_name)
                print(f"✓ Found train directory: {train_dir}")

        for dir_name in dirs:
            if "test" in dir_name.lower() and not any(x in dir_name.lower() for x in ["train", "val"]):
                test_dir = os.path.join(root, dir_name)
                print(f"✓ Found test directory: {test_dir}")

        for file in files:
            if "gt" in file.lower() and file.endswith((".mat", ".txt")):
                gt_file = os.path.join(root, file)
                print(f"✓ Found ground truth file: {gt_file}")

    # Check for image files
    train_images = 0
    test_images = 0

    if train_dir and os.path.exists(train_dir):
        train_images = len([f for f in os.listdir(train_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        print(f"✓ Found {train_images} training images")

    if test_dir and os.path.exists(test_dir):
        test_images = len([f for f in os.listdir(test_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        print(f"✓ Found {test_images} test images")

    # Create a verification report
    report = {
        "base_dir": output_dir,
        "train_dir": train_dir,
        "test_dir": test_dir,
        "gt_file": gt_file,
        "train_count": train_images,
        "test_count": test_images
    }

    print("\nDataset verification report:")
    print("="*50)
    for key, value in report.items():
        print(f"{key}: {value}")

    return report

if __name__ == "__main__":
    print("="*50)
    print("IIIT5K DATASET EXTRACTION AND VERIFICATION")
    print("="*50)

    # Check if the tar file exists
    tar_file = "IIIT5K-Word_V3.0.tar"
    if not os.path.exists(tar_file):
        print(f"\nERROR: '{tar_file}' not found in current directory!")
        print("Please place the IIIT5K-Word_V3.0.tar file in this directory and run again.")
        print("Current directory contents:")
        for item in os.listdir("."):
            print(f"- {item}")
        exit(1)

    # Extract and verify
    report = extract_and_verify_dataset(tar_file)

    if report:
        print("\n" + "="*50)
        print("NEXT STEPS FOR OCR TRAINING CODE")
        print("="*50)
        print("To use this dataset in the OCR training code, update the dataset paths as follows:")

        # Generate code snippet for dataset configuration
        print("\nIn your OCR training code, use these paths:")
        print("```python")
        print(f"# Dataset root directory")
        print(f"DATASET_ROOT = r'{os.path.abspath(report['base_dir'])}'")

        if report['train_dir']:
            print(f"TRAIN_DIR = r'{os.path.abspath(report['train_dir'])}'")
        if report['test_dir']:
            print(f"TEST_DIR = r'{os.path.abspath(report['test_dir'])}'")
        if report['gt_file']:
            print(f"GT_FILE = r'{os.path.abspath(report['gt_file'])}'")

        print("```")
        print("\nThis information will help configure the dataset loader correctly.")
        print("Based on common IIIT5K structure, here's how to implement the dataset loader:")

        # Provide specific dataset loader implementation
        print("\n```python")
        print("class IIIT5KDataset(Dataset):")
        print("    def __init__(self, root_dir, split='train', img_height=32, img_width=100):")
        print("        self.root_dir = root_dir")
        print("        self.img_height = img_height")
        print("        self.img_width = img_width")
        print("        ")
        print("        # Load ground truth")
        print("        gt_path = os.path.join(root_dir, 'gt.mat')  # Or the actual path from above")
        print("        # In practice, you'd use scipy.io.loadmat for .mat files")
        print("        # For demonstration, we'll assume a text-based GT")
        print("        ")
        print("        self.samples = []")
        print("        data_dir = os.path.join(root_dir, 'train' if split == 'train' else 'test')")
        print("        gt_file = os.path.join(root_dir, f'gt_{split}.txt')")
        print("        ")
        print("        # Parse ground truth file")
        print("        with open(gt_file, 'r') as f:")
        print("            for line in f:")
        print("                parts = line.strip().split(' ')")
        print("                img_name = parts[0]")
        print("                text = ' '.join(parts[1:])")
        print("                self.samples.append((os.path.join(data_dir, img_name), text))")
        print("        ")
        print("        # Character set for English OCR")
        print("        self.char_set = string.ascii_uppercase + string.digits + ' '")
        print("        self.char_to_idx = {char: idx for idx, char in enumerate(self.char_set)}")
        print("        self.idx_to_char = {idx: char for idx, char in enumerate(self.char_set)}")
        print("        ")
        print("        # Transformations")
        print("        self.transform = T.Compose([")
        print("            T.Resize((img_height, img_width)),")
        print("            T.Grayscale(),")
        print("            T.ToTensor(),")
        print("        ])")
        print("    ")
        print("    def __len__(self):")
        print("        return len(self.samples)")
        print("    ")
        print("    def __getitem__(self, idx):")
        print("        img_path, text = self.samples[idx]")
        print("        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)")
        print("        ")
        print("        # Convert text to indices")
        print("        text_indices = [self.char_to_idx.get(c.upper(), 0) for c in text if c.upper() in self.char_set]")
        print("        ")
        print("        # Convert to tensor")
        print("        img_tensor = self.transform(Image.fromarray(img))")
        print("        text_tensor = torch.tensor(text_indices, dtype=torch.long)")
        print("        ")
        print("        return img_tensor, text_tensor, len(text_indices)")
        print("```")


IIIT5K DATASET EXTRACTION AND VERIFICATION

ERROR: 'IIIT5K-Word_V3.0.tar' not found in current directory!
Please place the IIIT5K-Word_V3.0.tar file in this directory and run again.
Current directory contents:
- .config
- sample_data
Extracting IIIT5K-Word_V3.0.tar to iiit5k_dataset...
Error extracting file: [Errno 2] No such file or directory: 'IIIT5K-Word_V3.0.tar'


In [ ]:
# deep_ocr_custom_split.py
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import string
import scipy.io
import random

# ======================
# 1. MODIFIED DATASET CLASS WITH CUSTOM SPLIT
# ======================
class CustomSplitIIIT5KDataset(Dataset):
    def __init__(self, root_dir, split='train', img_height=32, img_width=128,
                 train_samples=4500, test_samples=500, random_seed=42):
        self.root_dir = root_dir
        self.split = split
        self.img_height = img_height
        self.img_width = img_width
        self.iiit5k_dir = os.path.join(root_dir, "IIIT5K")

        # Set random seed for reproducible splits
        random.seed(random_seed)
        np.random.seed(random_seed)

        # ✅ STEP 1: Define vocabulary FIRST (critical fix!)
        self.chars = string.ascii_letters + string.digits + "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
        self.char_to_idx = {char: idx + 1 for idx, char in enumerate(self.chars)}
        self.idx_to_char = {idx + 1: char for idx, char in enumerate(self.chars)}
        self.idx_to_char[0] = '<BLANK>'
        self.num_classes = len(self.chars) + 1  # 95 + 1

        print(f"Creating custom dataset split with {train_samples} train, {test_samples} test samples...")

        # Load all data from both original train and test sets
        all_samples = []

        # Load original train data
        train_mat_file = os.path.join(self.iiit5k_dir, "traindata.mat")
        train_data_dir = os.path.join(self.iiit5k_dir, "train")
        if os.path.exists(train_mat_file):
            all_samples.extend(self._load_mat_data(train_mat_file, train_data_dir, "train"))

        # Load original test data
        test_mat_file = os.path.join(self.iiit5k_dir, "testdata.mat")
        test_data_dir = os.path.join(self.iiit5k_dir, "test")
        if os.path.exists(test_mat_file):
            all_samples.extend(self._load_mat_data(test_mat_file, test_data_dir, "test"))

        print(f"Total samples loaded: {len(all_samples)}")

        # Shuffle all samples for random split
        random.shuffle(all_samples)

        # Create custom train/test split
        total_needed = train_samples + test_samples
        if len(all_samples) < total_needed:
            print(f"⚠️ Warning: Only {len(all_samples)} samples available, but {total_needed} requested")
            train_samples = min(train_samples, len(all_samples) - test_samples)
            test_samples = len(all_samples) - train_samples

        if split == 'train':
            self.samples = all_samples[:train_samples]
            print(f"✅ Created training set with {len(self.samples)} samples")
        else:  # test
            self.samples = all_samples[train_samples:train_samples + test_samples]
            print(f"✅ Created test set with {len(self.samples)} samples")

        if self.samples:
            sample_texts = [t for _, t in self.samples[:10]]
            lengths = [len(t) for _, t in self.samples]
            print(f"Sample texts: {sample_texts}")
            print(f"Text lengths: min={min(lengths)}, max={max(lengths)}, avg={np.mean(lengths):.1f}")
            print(f"Vocabulary size: {self.num_classes}")

        # Image transform
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((img_height, img_width), antialias=True),
            T.Grayscale(),
            T.ToTensor(),
            T.Normalize((0.5,), (0.5,))  # [-1, 1]
        ])

    def _load_mat_data(self, mat_file, data_dir, original_split):
        """Load data from .mat file and return list of (img_path, text) tuples"""
        samples = []

        try:
            mat_data = scipy.io.loadmat(mat_file)
        except Exception as e:
            print(f"❌ Failed to load {mat_file}: {e}")
            return samples

        data_key = f"{original_split}data"
        if data_key not in mat_data:
            available = [k for k in mat_data.keys() if not k.startswith('__')]
            print(f"❌ Key '{data_key}' not found in {mat_file}. Available: {available}")
            return samples

        data = mat_data[data_key]
        N = data.shape[1] if data.shape[0] == 1 else data.shape[0]
        print(f"Processing {N} samples from {original_split} data...")

        for i in range(N):
            try:
                sample = data[0, i] if data.shape[0] == 1 else data[i, 0]

                # Extract image path
                img_name = self._safe_extract_string(sample[0])
                if not img_name:
                    img_filename = f"sample_{i}.png"
                else:
                    img_filename = os.path.basename(img_name.strip())
                img_path = os.path.join(data_dir, img_filename)

                # Extract text
                text = self._safe_extract_string(sample[3])
                text = self._clean_text(text)

                if os.path.exists(img_path) and 1 <= len(text) <= 23:
                    samples.append((img_path, text))

            except Exception as e:
                continue

        return samples

    def _safe_extract_string(self, data):
        """Robustly extract string from .mat field (handles str, numeric array, object array)"""
        try:
            if isinstance(data, str):
                return data.strip()

            if isinstance(data, np.ndarray):
                # Case 1: Numeric array (ASCII codes)
                if np.issubdtype(data.dtype, np.number):
                    chars = []
                    for c in data.flatten():
                        try:
                            c_int = int(c)
                            if 32 <= c_int <= 126:  # Printable ASCII
                                chars.append(chr(c_int))
                        except (ValueError, TypeError):
                            continue
                    return ''.join(chars).strip()

                # Case 2: String array (U/S dtype)
                elif data.dtype.kind in ['U', 'S']:
                    return str(data.flatten()[0]).strip() if data.size > 0 else ""

                # Case 3: Object array (common in scipy.io.loadmat)
                elif data.dtype == np.object_:
                    item = data.item() if data.size == 1 else data[0]
                    if isinstance(item, str):
                        return item.strip()
                    return self._safe_extract_string(item)

            return str(data).strip()
        except Exception as e:
            return ""

    def _clean_text(self, text):
        """Keep only valid characters in full ASCII set"""
        return ''.join(c for c in text if c in self.chars).strip()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, text = self.samples[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️ Warning: Failed to load image {img_path}, using blank")
            img = np.ones((self.img_height, self.img_width), dtype=np.uint8) * 255

        text_indices = [self.char_to_idx[char] for char in text if char in self.char_to_idx]
        img_tensor = self.transform(img)
        return img_tensor, text_indices, text


# ======================
# 2. DEEP CRNN MODEL (UNCHANGED)
# ======================
class DeepCRNN(nn.Module):
    def __init__(self, img_height=32, num_classes=96, hidden_size=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
        )
        self.rnn = nn.LSTM(
            input_size=512,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=False,
            dropout=0.3
        )
        self.classifier = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(0.4)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        conv = self.cnn(x)  # [B, 512, 1, W]
        b, c, h, w = conv.size()
        conv = conv.view(b, c * h, w).permute(2, 0, 1)  # [W, B, 512]
        rnn_out, _ = self.rnn(conv)
        rnn_out = self.dropout(rnn_out)
        output = self.classifier(rnn_out)
        return F.log_softmax(output, dim=2)


# ======================
# 3. CTC UTILITIES (UNCHANGED)
# ======================
def ctc_collate_fn(batch):
    images, text_indices_list, raw_texts = zip(*batch)
    images = torch.stack(images, 0)
    targets = []
    target_lengths = []
    for text_indices in text_indices_list:
        if len(text_indices) > 0:
            targets.extend(text_indices)
            target_lengths.append(len(text_indices))
        else:
            targets.append(0)
            target_lengths.append(1)
    return images, torch.tensor(targets), torch.tensor(target_lengths), raw_texts

def ctc_decode(log_probs, idx_to_char, blank_idx=0):
    """Greedy CTC decoding"""
    seq_len, batch_size, num_classes = log_probs.shape
    predictions = []
    for b in range(batch_size):
        pred_indices = log_probs[:, b, :].argmax(dim=1).cpu().numpy()
        decoded = []
        prev_idx = blank_idx
        for idx in pred_indices:
            if idx != blank_idx and idx != prev_idx:
                if idx in idx_to_char and idx_to_char[idx] != '<BLANK>':
                    decoded.append(idx_to_char[idx])
            prev_idx = idx
        predictions.append(''.join(decoded))
    return predictions

def calculate_metrics(pred_texts, true_texts):
    """Calculate word and character accuracy"""
    if not pred_texts or not true_texts:
        return 0.0, 0.0
    word_acc = sum(p == t for p, t in zip(pred_texts, true_texts)) / len(true_texts)
    total_chars = sum(max(len(p), len(t)) for p, t in zip(pred_texts, true_texts))
    correct_chars = sum(p[i] == t[i] for p, t in zip(pred_texts, true_texts) for i in range(min(len(p), len(t))))
    char_acc = correct_chars / total_chars if total_chars > 0 else 0.0
    return word_acc, char_acc


# ======================
# 4. TRAINING LOOP WITH CUSTOM SPLIT
# ======================
def train_deep_ocr_custom_split():
    print("🚀 DEEP OCR TRAINING WITH CUSTOM SPLIT (4500 train, 500 test)")
    print("=" * 60)

    # CONFIGURATION - Updated based on your dataset structure
    dataset_root = "/content/iiit5k_dataset"  # Your extracted dataset path

    # Verify dataset exists
    print(f"🔍 Checking dataset root: {dataset_root}")
    print(f"📁 Path exists: {os.path.exists(dataset_root)}")

    if not os.path.exists(dataset_root):
        # Try alternative paths
        alternative_paths = ["./iiit5k_dataset", "/content/iiit5k_dataset"]
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                dataset_root = alt_path
                print(f"✅ Found dataset at alternative path: {dataset_root}")
                break
        else:
            print(f"❌ Dataset root not found at any location")
            print("💡 Checked paths:")
            for path in ["/content/iiit5k_dataset"] + alternative_paths:
                print(f"   {path}: {'✅' if os.path.exists(path) else '❌'}")
            return None
    else:
        print(f"✅ Dataset root found: {dataset_root}")

    # Verify IIIT5K subdirectory structure
    iiit5k_path = os.path.join(dataset_root, "IIIT5K")
    print(f"🔍 Checking IIIT5K path: {iiit5k_path}")
    print(f"📁 IIIT5K exists: {os.path.exists(iiit5k_path)}")

    if os.path.exists(iiit5k_path):
        contents = os.listdir(iiit5k_path)
        print(f"📊 IIIT5K contents: {contents}")

        # Check required files
        required_items = ['train', 'test', 'traindata.mat', 'testdata.mat']
        missing_items = [item for item in required_items if item not in contents]

        if missing_items:
            print(f"❌ Missing required items: {missing_items}")
            return None
        else:
            print("✅ All required dataset components found!")
    else:
        print(f"❌ IIIT5K subdirectory not found at: {iiit5k_path}")
        return None

    batch_size = 32
    num_epochs = 250
    learning_rate = 1e-3
    train_samples = 4500
    test_samples = 500

    print(f"Custom split: {train_samples} train, {test_samples} test")
    print(f"Batch size: {batch_size}")
    print(f"Epochs: {num_epochs}")
    print(f"Learning rate: {learning_rate}")

    # Load datasets with custom split
    print("\nCreating custom split datasets...")
    try:
        train_dataset = CustomSplitIIIT5KDataset(
            dataset_root, 'train', 32, 128, train_samples, test_samples
        )
        test_dataset = CustomSplitIIIT5KDataset(
            dataset_root, 'test', 32, 128, train_samples, test_samples
        )
    except Exception as e:
        print(f"❌ Dataset creation failed: {e}")
        return None

    if len(train_dataset) == 0 or len(test_dataset) == 0:
        print("❌ No valid data loaded.")
        return None

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              collate_fn=ctc_collate_fn, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                             collate_fn=ctc_collate_fn, num_workers=0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nDevice: {device}")

    model = DeepCRNN(32, train_dataset.num_classes).to(device)
    criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Training batches: {len(train_loader)}")
    print(f"Test batches: {len(test_loader)}")

    best_word_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_word_acc = 0.0
        num_train_batches = 0

        print(f"\n📈 Epoch {epoch+1}/{num_epochs}")
        pbar = tqdm(train_loader, desc="Training")
        for images, targets, target_lengths, raw_texts in pbar:
            images = images.to(device)
            targets = targets.to(device)
            target_lengths = target_lengths.to(device)

            log_probs = model(images)
            input_lengths = torch.full((images.size(0),), log_probs.size(0), device=device, dtype=torch.long)
            loss = criterion(log_probs, targets, input_lengths, target_lengths)

            optimizer.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()

            # Log every 20 batches (adjusted for potentially more batches)
            if num_train_batches % 20 == 0:
                pred_texts = ctc_decode(log_probs.cpu(), train_dataset.idx_to_char)
                word_acc, _ = calculate_metrics(pred_texts, raw_texts)
                train_word_acc += word_acc
                pbar.set_postfix({"loss": f"{loss.item():.3f}", "word_acc": f"{word_acc:.3f}"})

            num_train_batches += 1

        scheduler.step()
        train_loss /= num_train_batches
        train_word_acc /= (num_train_batches // 20 + 1)

        # Validation
        model.eval()
        val_preds, val_truths = [], []
        with torch.no_grad():
            for images, targets, target_lengths, raw_texts in test_loader:
                images = images.to(device)
                log_probs = model(images)
                pred_texts = ctc_decode(log_probs.cpu(), test_dataset.idx_to_char)
                val_preds.extend(pred_texts)
                val_truths.extend(raw_texts)

        val_word_acc, val_char_acc = calculate_metrics(val_preds, val_truths)

        print(f"\n📊 Epoch {epoch+1} Summary:")
        print(f"  Training Loss: {train_loss:.4f}")
        print(f"  Training Word Acc: {train_word_acc:.4f}")
        print(f"  Validation Word Acc: {val_word_acc:.4f}")
        print(f"  Validation Char Acc: {val_char_acc:.4f}")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

        if val_word_acc > best_word_acc:
            best_word_acc = val_word_acc
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'val_word_acc': val_word_acc,
                'idx_to_char': train_dataset.idx_to_char,
                'num_classes': train_dataset.num_classes
            }, 'deep_ocr_custom_split_model.pth')
            print(f"  ✅ New best model saved! Word Acc: {best_word_acc:.4f}")

        # Sample predictions
        print(f"\n🎯 Sample predictions:")
        indices = np.random.choice(len(val_preds), min(5, len(val_preds)), replace=False)
        for i in indices:
            print(f"  True: '{val_truths[i]}' | Pred: '{val_preds[i]}'")

    print(f"\n🎉 Training completed!")
    print(f"Best validation word accuracy: {best_word_acc:.4f}")
    return best_word_acc


# ======================
# 5. MAIN EXECUTION
# ======================
if __name__ == "__main__":
    print("⚡ DEEP OCR SYSTEM WITH CUSTOM SPLIT")
    print("=" * 50)
    choice = input("\nChoose option:\n1. Train with custom split (4500 train, 500 test)\n2. Exit\nChoice: ")
    if choice == "1":
        accuracy = train_deep_ocr_custom_split()
        if accuracy is not None:
            print(f"\n🏆 Final result: {accuracy:.4f} word accuracy")
        else:
            print("\n❌ Training failed. Check error messages above.")
    else:
        print("Goodbye! 👋")

⚡ DEEP OCR SYSTEM WITH CUSTOM SPLIT

Choose option:
1. Train with custom split (4500 train, 500 test)
2. Exit
Choice: 1
🚀 DEEP OCR TRAINING WITH CUSTOM SPLIT (4500 train, 500 test)
🔍 Checking dataset root: /content/iiit5k_dataset
📁 Path exists: True
✅ Dataset root found: /content/iiit5k_dataset
🔍 Checking IIIT5K path: /content/iiit5k_dataset/IIIT5K
📁 IIIT5K exists: True
📊 IIIT5K contents: ['test', 'traindata.mat', 'trainCharBound.mat', 'testCharBound.mat', 'train', 'README', 'lexicon.txt', 'testdata.mat']
✅ All required dataset components found!
Custom split: 4500 train, 500 test
Batch size: 32
Epochs: 250
Learning rate: 0.001

Creating custom split datasets...
Creating custom dataset split with 4500 train, 500 test samples...
Processing 2000 samples from train data...
Processing 3000 samples from test data...
Total samples loaded: 5000
✅ Created training set with 4500 samples
Sample texts: ['YOU', 'SBI', 'MEDICINE', 'MARRY', 'YOURSELF', 'CAN', 'THIS', '21', 'VOLTS', 'WAVERLEY']
Text l

Training: 100%|██████████| 141/141 [00:10<00:00, 13.73it/s, loss=3.593, word_acc=0.000]



📊 Epoch 1 Summary:
  Training Loss: 4.1951
  Training Word Acc: 0.0000
  Validation Word Acc: 0.0000
  Validation Char Acc: 0.0000
  Learning Rate: 0.001000

🎯 Sample predictions:
  True: 'SALT' | Pred: ''
  True: 'BIG' | Pred: ''
  True: 'DANGERS' | Pred: ''
  True: 'THE' | Pred: ''
  True: 'INVERCARGILL' | Pred: ''

📈 Epoch 2/250


Training: 100%|██████████| 141/141 [00:08<00:00, 17.10it/s, loss=3.409, word_acc=0.000]



📊 Epoch 2 Summary:
  Training Loss: 3.5359
  Training Word Acc: 0.0000
  Validation Word Acc: 0.0000
  Validation Char Acc: 0.0000
  Learning Rate: 0.001000

🎯 Sample predictions:
  True: 'AN' | Pred: ''
  True: 'WISTERIA' | Pred: ''
  True: 'MELIN' | Pred: ''
  True: 'EASTWOOD' | Pred: ''
  True: 'PRODAJA' | Pred: ''

📈 Epoch 3/250


...


📊 Epoch 248 Summary:
  Training Loss: 0.0001
  Training Word Acc: 1.0000
  Validation Word Acc: 0.7600
  Validation Char Acc: 0.8428
  Learning Rate: 0.000000

🎯 Sample predictions:
  True: 'NOW' | Pred: 'NAY'
  True: 'STATE' | Pred: 'STATE'
  True: 'BLUE' | Pred: 'BLUE'
  True: 'SBI' | Pred: 'SBR'
  True: '2ND' | Pred: '2ND'

📈 Epoch 249/250


Training: 100%|██████████| 141/141 [00:08<00:00, 16.61it/s, loss=0.000, word_acc=1.000]



📊 Epoch 249 Summary:
  Training Loss: 0.0001
  Training Word Acc: 1.0000
  Validation Word Acc: 0.7600
  Validation Char Acc: 0.8391
  Learning Rate: 0.000000

🎯 Sample predictions:
  True: 'LEAGUE' | Pred: 'LEAGUE'
  True: 'NOT' | Pred: 'NOT'
  True: 'DREAM' | Pred: 'DREAM'
  True: 'POD' | Pred: 'POD'
  True: 'HOME' | Pred: 'HOME'

📈 Epoch 250/250


Training: 100%|██████████| 141/141 [00:08<00:00, 16.11it/s, loss=0.000, word_acc=1.000]



📊 Epoch 250 Summary:
  Training Loss: 0.0000
  Training Word Acc: 1.0000
  Validation Word Acc: 0.7620
  Validation Char Acc: 0.8391
  Learning Rate: 0.000000

🎯 Sample predictions:
  True: 'JALAN' | Pred: 'JALAN'
  True: 'BACK' | Pred: 'BACK'
  True: 'CUSTOMERS' | Pred: 'CISTOMERS'
  True: 'PEACE' | Pred: 'REACE'
  True: 'ROOM' | Pred: 'POORI'

🎉 Training completed!
Best validation word accuracy: 0.7680

🏆 Final result: 0.7680 word accuracy


In [ ]:
# enhanced_deep_ocr.py
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import string
import scipy.io
import random
import math
from torch.cuda.amp import autocast, GradScaler

# ======================
# 1. ENHANCED DATASET WITH AUGMENTATION
# ======================
class EnhancedCustomSplitIIIT5KDataset(Dataset):
    def __init__(self, root_dir, split='train', img_height=32, img_width=128,
                 train_samples=4500, test_samples=500, random_seed=42, augment=True):
        self.root_dir = root_dir
        self.split = split
        self.img_height = img_height
        self.img_width = img_width
        self.iiit5k_dir = os.path.join(root_dir, "IIIT5K")
        self.augment = augment and split == 'train'

        # Set random seed for reproducible splits
        random.seed(random_seed)
        np.random.seed(random_seed)

        # Define vocabulary
        self.chars = string.ascii_letters + string.digits + "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
        self.char_to_idx = {char: idx + 1 for idx, char in enumerate(self.chars)}
        self.idx_to_char = {idx + 1: char for idx, char in enumerate(self.chars)}
        self.idx_to_char[0] = '<BLANK>'
        self.num_classes = len(self.chars) + 1  # 95 + 1

        print(f"Creating custom dataset split with {train_samples} train, {test_samples} test samples...")

        # Load all data from both original train and test sets
        all_samples = []

        # Load original train data
        train_mat_file = os.path.join(self.iiit5k_dir, "traindata.mat")
        train_data_dir = os.path.join(self.iiit5k_dir, "train")
        if os.path.exists(train_mat_file):
            all_samples.extend(self._load_mat_data(train_mat_file, train_data_dir, "train"))

        # Load original test data
        test_mat_file = os.path.join(self.iiit5k_dir, "testdata.mat")
        test_data_dir = os.path.join(self.iiit5k_dir, "test")
        if os.path.exists(test_mat_file):
            all_samples.extend(self._load_mat_data(test_mat_file, test_data_dir, "test"))

        print(f"Total samples loaded: {len(all_samples)}")

        # Shuffle all samples for random split
        random.shuffle(all_samples)

        # Create custom train/test split
        total_needed = train_samples + test_samples
        if len(all_samples) < total_needed:
            print(f"⚠️ Warning: Only {len(all_samples)} samples available, but {total_needed} requested")
            train_samples = min(train_samples, len(all_samples) - test_samples)
            test_samples = len(all_samples) - train_samples

        if split == 'train':
            self.samples = all_samples[:train_samples]
            print(f"✅ Created training set with {len(self.samples)} samples")
        else:  # test
            self.samples = all_samples[train_samples:train_samples + test_samples]
            print(f"✅ Created test set with {len(self.samples)} samples")

        if self.samples:
            sample_texts = [t for _, t in self.samples[:10]]
            lengths = [len(t) for _, t in self.samples]
            print(f"Sample texts: {sample_texts}")
            print(f"Text lengths: min={min(lengths)}, max={max(lengths)}, avg={np.mean(lengths):.1f}")
            print(f"Vocabulary size: {self.num_classes}")

        # Enhanced transforms with augmentation
        if self.augment:
            self.transform = T.Compose([
                T.ToPILImage(),
                T.Resize((img_height, img_width), antialias=True),
                T.Grayscale(),
                # Data augmentation
                T.RandomApply([T.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))], p=0.3),
                T.ColorJitter(brightness=0.3, contrast=0.3),
                T.RandomApply([T.RandomRotation(degrees=2)], p=0.2),
                T.ToTensor(),
                T.Normalize((0.5,), (0.5,))  # [-1, 1]
            ])
        else:
            self.transform = T.Compose([
                T.ToPILImage(),
                T.Resize((img_height, img_width), antialias=True),
                T.Grayscale(),
                T.ToTensor(),
                T.Normalize((0.5,), (0.5,))  # [-1, 1]
            ])

    def _load_mat_data(self, mat_file, data_dir, original_split):
        """Load data from .mat file and return list of (img_path, text) tuples"""
        samples = []

        try:
            mat_data = scipy.io.loadmat(mat_file)
        except Exception as e:
            print(f"❌ Failed to load {mat_file}: {e}")
            return samples

        data_key = f"{original_split}data"
        if data_key not in mat_data:
            available = [k for k in mat_data.keys() if not k.startswith('__')]
            print(f"❌ Key '{data_key}' not found in {mat_file}. Available: {available}")
            return samples

        data = mat_data[data_key]
        N = data.shape[1] if data.shape[0] == 1 else data.shape[0]
        print(f"Processing {N} samples from {original_split} data...")

        for i in range(N):
            try:
                sample = data[0, i] if data.shape[0] == 1 else data[i, 0]

                # Extract image path
                img_name = self._safe_extract_string(sample[0])
                if not img_name:
                    img_filename = f"sample_{i}.png"
                else:
                    img_filename = os.path.basename(img_name.strip())
                img_path = os.path.join(data_dir, img_filename)

                # Extract text
                text = self._safe_extract_string(sample[3])
                text = self._clean_text(text)

                if os.path.exists(img_path) and 1 <= len(text) <= 23:
                    samples.append((img_path, text))

            except Exception as e:
                continue

        return samples

    def _safe_extract_string(self, data):
        """Robustly extract string from .mat field (handles str, numeric array, object array)"""
        try:
            if isinstance(data, str):
                return data.strip()

            if isinstance(data, np.ndarray):
                # Case 1: Numeric array (ASCII codes)
                if np.issubdtype(data.dtype, np.number):
                    chars = []
                    for c in data.flatten():
                        try:
                            c_int = int(c)
                            if 32 <= c_int <= 126:  # Printable ASCII
                                chars.append(chr(c_int))
                        except (ValueError, TypeError):
                            continue
                    return ''.join(chars).strip()

                # Case 2: String array (U/S dtype)
                elif data.dtype.kind in ['U', 'S']:
                    return str(data.flatten()[0]).strip() if data.size > 0 else ""

                # Case 3: Object array (common in scipy.io.loadmat)
                elif data.dtype == np.object_:
                    item = data.item() if data.size == 1 else data[0]
                    if isinstance(item, str):
                        return item.strip()
                    return self._safe_extract_string(item)

            return str(data).strip()
        except Exception as e:
            return ""

    def _clean_text(self, text):
        """Keep only valid characters in full ASCII set"""
        return ''.join(c for c in text if c in self.chars).strip()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, text = self.samples[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️ Warning: Failed to load image {img_path}, using blank")
            img = np.ones((self.img_height, self.img_width), dtype=np.uint8) * 255

        # Additional noise augmentation for training
        if self.augment and random.random() < 0.2:
            noise = np.random.normal(0, 10, img.shape).astype(np.uint8)
            img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

        text_indices = [self.char_to_idx[char] for char in text if char in self.char_to_idx]
        img_tensor = self.transform(img)
        return img_tensor, text_indices, text


# ======================
# 2. ENHANCED CRNN MODEL WITH ATTENTION
# ======================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class EnhancedDeepCRNN(nn.Module):
    def __init__(self, img_height=32, num_classes=96, hidden_size=256, dropout=0.3):
        super().__init__()

        # Enhanced CNN backbone with residual connections
        self.cnn = nn.Sequential(
            # Stage 1
            nn.Conv2d(1, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.1),

            # Stage 2
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.1),

            # Stage 3
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.MaxPool2d((2, 1), (2, 1)),
            nn.Dropout2d(0.1),

            # Stage 4
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d((2, 1), (2, 1)),
            nn.Dropout2d(0.1),

            # Stage 5
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.MaxPool2d((2, 1), (2, 1)),
        )

        # Enhanced RNN with more layers and higher capacity
        self.rnn = nn.LSTM(
            input_size=512,
            hidden_size=hidden_size,
            num_layers=3,  # Increased from 2
            bidirectional=True,
            batch_first=False,
            dropout=dropout
        )

        # Attention mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,
            num_heads=8,
            dropout=dropout,
            batch_first=False
        )

        # Positional encoding for attention
        self.pos_encoding = PositionalEncoding(hidden_size * 2)

        # Classification layers with residual connection
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )

        self.dropout = nn.Dropout(dropout)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.LSTM):
                for name, param in m.named_parameters():
                    if 'weight' in name:
                        nn.init.xavier_uniform_(param)
                    elif 'bias' in name:
                        nn.init.constant_(param, 0)

    def forward(self, x):
        # CNN feature extraction
        conv = self.cnn(x)  # [B, 512, 1, W]
        b, c, h, w = conv.size()
        conv = conv.view(b, c * h, w).permute(2, 0, 1)  # [W, B, 512]

        # RNN processing
        rnn_out, _ = self.rnn(conv)  # [W, B, hidden_size*2]
        rnn_out = self.dropout(rnn_out)

        # Add positional encoding and apply attention
        rnn_out_pe = self.pos_encoding(rnn_out)
        attended_out, _ = self.attention(rnn_out_pe, rnn_out_pe, rnn_out_pe)

        # Residual connection
        attended_out = attended_out + rnn_out
        attended_out = self.dropout(attended_out)

        # Classification
        output = self.classifier(attended_out)
        return F.log_softmax(output, dim=2)


# ======================
# 3. ENHANCED CTC UTILITIES WITH BEAM SEARCH
# ======================
def ctc_collate_fn(batch):
    images, text_indices_list, raw_texts = zip(*batch)
    images = torch.stack(images, 0)
    targets = []
    target_lengths = []
    for text_indices in text_indices_list:
        if len(text_indices) > 0:
            targets.extend(text_indices)
            target_lengths.append(len(text_indices))
        else:
            targets.append(0)
            target_lengths.append(1)
    return images, torch.tensor(targets), torch.tensor(target_lengths), raw_texts

def beam_search_decode(log_probs, idx_to_char, beam_width=5, blank_idx=0):
    """Beam search CTC decoding for better accuracy"""
    seq_len, batch_size, num_classes = log_probs.shape
    predictions = []

    for b in range(batch_size):
        probs = log_probs[:, b, :].exp().cpu().numpy()  # Convert to probabilities

        # Initialize beam
        beam = [([], 0.0)]  # (sequence, log_prob)

        for t in range(seq_len):
            new_beam = []

            for seq, log_prob in beam:
                for c in range(num_classes):
                    new_log_prob = log_prob + np.log(probs[t, c] + 1e-8)

                    if c == blank_idx:
                        # Blank - no character added
                        new_beam.append((seq, new_log_prob))
                    else:
                        # Character
                        if len(seq) == 0 or seq[-1] != c:
                            # New character or different from previous
                            new_seq = seq + [c]
                            new_beam.append((new_seq, new_log_prob))
                        else:
                            # Same as previous - don't add
                            new_beam.append((seq, new_log_prob))

            # Keep top beam_width candidates
            beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_width]

        # Get best sequence
        best_seq = beam[0][0]
        decoded_text = ''.join([idx_to_char.get(idx, '') for idx in best_seq if idx in idx_to_char])
        predictions.append(decoded_text)

    return predictions

def ctc_decode(log_probs, idx_to_char, blank_idx=0, use_beam_search=False, beam_width=5):
    """Enhanced CTC decoding with optional beam search"""
    if use_beam_search:
        return beam_search_decode(log_probs, idx_to_char, beam_width, blank_idx)

    # Standard greedy decoding
    seq_len, batch_size, num_classes = log_probs.shape
    predictions = []
    for b in range(batch_size):
        pred_indices = log_probs[:, b, :].argmax(dim=1).cpu().numpy()
        decoded = []
        prev_idx = blank_idx
        for idx in pred_indices:
            if idx != blank_idx and idx != prev_idx:
                if idx in idx_to_char and idx_to_char[idx] != '<BLANK>':
                    decoded.append(idx_to_char[idx])
            prev_idx = idx
        predictions.append(''.join(decoded))
    return predictions

def calculate_metrics(pred_texts, true_texts):
    """Calculate word and character accuracy"""
    if not pred_texts or not true_texts:
        return 0.0, 0.0
    word_acc = sum(p == t for p, t in zip(pred_texts, true_texts)) / len(true_texts)
    total_chars = sum(max(len(p), len(t)) for p, t in zip(pred_texts, true_texts))
    if total_chars == 0:
        return word_acc, 0.0
    correct_chars = sum(p[i] == t[i] for p, t in zip(pred_texts, true_texts) for i in range(min(len(p), len(t))))
    char_acc = correct_chars / total_chars
    return word_acc, char_acc


# ======================
# 4. ENHANCED TRAINING WITH EARLY STOPPING AND MIXED PRECISION
# ======================
class EarlyStopping:
    def __init__(self, patience=20, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_score
            self.counter = 0

def train_enhanced_deep_ocr():
    print("🚀 ENHANCED DEEP OCR TRAINING WITH IMPROVEMENTS")
    print("=" * 60)

    # CONFIGURATION
    dataset_root = "/content/iiit5k_dataset"

    # Verify dataset exists
    print(f"🔍 Checking dataset root: {dataset_root}")
    print(f"📁 Path exists: {os.path.exists(dataset_root)}")

    if not os.path.exists(dataset_root):
        alternative_paths = ["./iiit5k_dataset", "/content/iiit5k_dataset"]
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                dataset_root = alt_path
                print(f"✅ Found dataset at alternative path: {dataset_root}")
                break
        else:
            print(f"❌ Dataset root not found at any location")
            return None
    else:
        print(f"✅ Dataset root found: {dataset_root}")

    # Verify IIIT5K subdirectory structure
    iiit5k_path = os.path.join(dataset_root, "IIIT5K")
    print(f"🔍 Checking IIIT5K path: {iiit5k_path}")
    print(f"📁 IIIT5K exists: {os.path.exists(iiit5k_path)}")

    if os.path.exists(iiit5k_path):
        contents = os.listdir(iiit5k_path)
        print(f"📊 IIIT5K contents: {contents}")
        required_items = ['train', 'test', 'traindata.mat', 'testdata.mat']
        missing_items = [item for item in required_items if item not in contents]
        if missing_items:
            print(f"❌ Missing required items: {missing_items}")
            return None
        else:
            print("✅ All required dataset components found!")
    else:
        print(f"❌ IIIT5K subdirectory not found at: {iiit5k_path}")
        return None

    # Enhanced hyperparameters
    batch_size = 32
    max_epochs = 200  # Reduced since we have early stopping
    learning_rate = 2e-3  # Slightly higher for OneCycle
    train_samples = 4500
    test_samples = 500
    hidden_size = 256  # Increased capacity
    dropout = 0.3

    print(f"Enhanced configuration:")
    print(f"  Custom split: {train_samples} train, {test_samples} test")
    print(f"  Batch size: {batch_size}")
    print(f"  Max epochs: {max_epochs}")
    print(f"  Learning rate: {learning_rate}")
    print(f"  Hidden size: {hidden_size}")
    print(f"  Dropout: {dropout}")

    # Load enhanced datasets with augmentation
    print("\nCreating enhanced datasets with augmentation...")
    try:
        train_dataset = EnhancedCustomSplitIIIT5KDataset(
            dataset_root, 'train', 32, 128, train_samples, test_samples, augment=True
        )
        test_dataset = EnhancedCustomSplitIIIT5KDataset(
            dataset_root, 'test', 32, 128, train_samples, test_samples, augment=False
        )
    except Exception as e:
        print(f"❌ Dataset creation failed: {e}")
        return None

    if len(train_dataset) == 0 or len(test_dataset) == 0:
        print("❌ No valid data loaded.")
        return None

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              collate_fn=ctc_collate_fn, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                             collate_fn=ctc_collate_fn, num_workers=2, pin_memory=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nDevice: {device}")

    # Enhanced model
    model = EnhancedDeepCRNN(32, train_dataset.num_classes, hidden_size, dropout).to(device)
    criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)

    # OneCycle learning rate scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = OneCycleLR(optimizer, max_lr=learning_rate,
                          steps_per_epoch=len(train_loader), epochs=max_epochs)

    # Mixed precision training
    scaler = GradScaler()

    # Early stopping
    early_stopping = EarlyStopping(patience=25, min_delta=0.001)

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Training batches: {len(train_loader)}")
    print(f"Test batches: {len(test_loader)}")

    best_word_acc = 0.0
    patience_counter = 0

    for epoch in range(max_epochs):
        model.train()
        train_loss = 0.0
        train_word_acc = 0.0
        num_train_batches = 0

        print(f"\n📈 Epoch {epoch+1}/{max_epochs}")
        pbar = tqdm(train_loader, desc="Training")

        for images, targets, target_lengths, raw_texts in pbar:
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            target_lengths = target_lengths.to(device, non_blocking=True)

            optimizer.zero_grad()

            # Mixed precision forward pass
            with autocast():
                log_probs = model(images)
                input_lengths = torch.full((images.size(0),), log_probs.size(0),
                                         device=device, dtype=torch.long)
                loss = criterion(log_probs, targets, input_lengths, target_lengths)

            # Mixed precision backward pass
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            train_loss += loss.item()

            # Calculate accuracy every 20 batches
            if num_train_batches % 20 == 0:
                pred_texts = ctc_decode(log_probs.cpu(), train_dataset.idx_to_char)
                word_acc, _ = calculate_metrics(pred_texts, raw_texts)
                train_word_acc += word_acc
                pbar.set_postfix({"loss": f"{loss.item():.3f}", "word_acc": f"{word_acc:.3f}"})

            num_train_batches += 1

        train_loss /= num_train_batches
        train_word_acc /= (num_train_batches // 20 + 1)

        # Validation with beam search
        model.eval()
        val_preds, val_truths = [], []
        with torch.no_grad():
            for images, targets, target_lengths, raw_texts in test_loader:
                images = images.to(device, non_blocking=True)
                with autocast():
                    log_probs = model(images)
                # Use beam search for validation
                pred_texts = ctc_decode(log_probs.cpu(), test_dataset.idx_to_char,
                                      use_beam_search=True, beam_width=3)
                val_preds.extend(pred_texts)
                val_truths.extend(raw_texts)

        val_word_acc, val_char_acc = calculate_metrics(val_preds, val_truths)

        print(f"\n📊 Epoch {epoch+1} Summary:")
        print(f"  Training Loss: {train_loss:.4f}")
        print(f"  Training Word Acc: {train_word_acc:.4f}")
        print(f"  Validation Word Acc: {val_word_acc:.4f}")
        print(f"  Validation Char Acc: {val_char_acc:.4f}")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

        # Save best model
        if val_word_acc > best_word_acc:
            best_word_acc = val_word_acc
            patience_counter = 0
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_word_acc': val_word_acc,
                'val_char_acc': val_char_acc,
                'idx_to_char': train_dataset.idx_to_char,
                'num_classes': train_dataset.num_classes,
                'hidden_size': hidden_size
            }, 'enhanced_deep_ocr_model.pth')
            print(f"  ✅ New best model saved! Word Acc: {best_word_acc:.4f}")
        else:
            patience_counter += 1

        # Early stopping check
        early_stopping(val_word_acc)
        if early_stopping.early_stop:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break

        # Sample predictions
        print(f"\n🎯 Sample predictions (with beam search):")
        indices = np.random.choice(len(val_preds), min(5, len(val_preds)), replace=False)
        for i in indices:
            print(f"  True: '{val_truths[i]}' | Pred: '{val_preds[i]}'")

    print(f"\n🎉 Training completed!")
    print(f"Best validation word accuracy: {best_word_acc:.4f}")
    return best_word_acc


# ======================
# 5. MODEL INFERENCE WITH BEAM SEARCH
# ======================
def load_and_test_model(model_path, test_images_dir=None):
    """Load trained model and test on new images"""
    print("🔍 Loading trained model...")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    checkpoint = torch.load(model_path, map_location=device)

    # Recreate model
    model = EnhancedDeepCRNN(
        img_height=32,
        num_classes=checkpoint['num_classes'],
        hidden_size=checkpoint.get('hidden_size', 256)
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    idx_to_char = checkpoint['idx_to_char']

    print(f"✅ Model loaded successfully!")
    print(f"   Validation Word Accuracy: {checkpoint['val_word_acc']:.4f}")
    print(f"   Validation Char Accuracy: {checkpoint['val_char_acc']:.4f}")

    # Transform for inference
    transform = T.Compose([
        T.ToPILImage(),
        T.Resize((32, 128), antialias=True),
        T.Grayscale(),
        T.ToTensor(),
        T.Normalize((0.5,), (0.5,))
    ])

    def predict_text(image_path):
        """Predict text from single image"""
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return "Error: Could not load image"

        img_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            with autocast():
                log_probs = model(img_tensor)

            # Use beam search for better accuracy
            pred_texts = ctc_decode(log_probs.cpu(), idx_to_char,
                                  use_beam_search=True, beam_width=5)
            return pred_texts[0]

    return predict_text


# ======================
# 6. ADVANCED EVALUATION METRICS
# ======================
def detailed_evaluation(model, test_loader, idx_to_char, device):
    """Comprehensive model evaluation with detailed metrics"""
    model.eval()
    all_preds = []
    all_truths = []
    char_confusion = {}
    length_accuracy = {i: {'correct': 0, 'total': 0} for i in range(1, 21)}

    print("🔍 Running detailed evaluation...")

    with torch.no_grad():
        for images, targets, target_lengths, raw_texts in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device, non_blocking=True)

            with autocast():
                log_probs = model(images)

            # Get predictions with both greedy and beam search
            greedy_preds = ctc_decode(log_probs.cpu(), idx_to_char, use_beam_search=False)
            beam_preds = ctc_decode(log_probs.cpu(), idx_to_char, use_beam_search=True, beam_width=5)

            all_preds.extend(beam_preds)
            all_truths.extend(raw_texts)

            # Length-based accuracy
            for pred, truth in zip(beam_preds, raw_texts):
                length = min(len(truth), 20)
                length_accuracy[length]['total'] += 1
                if pred == truth:
                    length_accuracy[length]['correct'] += 1

            # Character-level confusion analysis
            for pred, truth in zip(beam_preds, raw_texts):
                for i, (p_char, t_char) in enumerate(zip(pred, truth)):
                    if t_char not in char_confusion:
                        char_confusion[t_char] = {'correct': 0, 'total': 0, 'confused_with': {}}
                    char_confusion[t_char]['total'] += 1
                    if p_char == t_char:
                        char_confusion[t_char]['correct'] += 1
                    else:
                        if p_char not in char_confusion[t_char]['confused_with']:
                            char_confusion[t_char]['confused_with'][p_char] = 0
                        char_confusion[t_char]['confused_with'][p_char] += 1

    # Calculate overall metrics
    word_acc, char_acc = calculate_metrics(all_preds, all_truths)

    print(f"\n📊 Detailed Evaluation Results:")
    print(f"   Overall Word Accuracy: {word_acc:.4f}")
    print(f"   Overall Character Accuracy: {char_acc:.4f}")

    # Length-based accuracy
    print(f"\n📏 Accuracy by text length:")
    for length in range(1, 11):  # Show first 10 lengths
        if length_accuracy[length]['total'] > 0:
            acc = length_accuracy[length]['correct'] / length_accuracy[length]['total']
            print(f"   Length {length}: {acc:.3f} ({length_accuracy[length]['correct']}/{length_accuracy[length]['total']})")

    # Most confused characters
    print(f"\n🔤 Most problematic characters:")
    char_errors = []
    for char, stats in char_confusion.items():
        if stats['total'] > 5:  # Only consider characters that appear frequently
            error_rate = 1 - (stats['correct'] / stats['total'])
            char_errors.append((char, error_rate, stats['total']))

    char_errors.sort(key=lambda x: x[1], reverse=True)
    for char, error_rate, total in char_errors[:10]:
        print(f"   '{char}': {error_rate:.3f} error rate ({total} samples)")
        # Show top confusions
        if char in char_confusion and char_confusion[char]['confused_with']:
            top_confusions = sorted(char_confusion[char]['confused_with'].items(),
                                  key=lambda x: x[1], reverse=True)[:3]
            confusion_str = ', '.join([f"'{conf_char}'({count})" for conf_char, count in top_confusions])
            print(f"      → Often confused with: {confusion_str}")

    return word_acc, char_acc


# ======================
# 7. MAIN EXECUTION WITH ENHANCED OPTIONS
# ======================
if __name__ == "__main__":
    print("⚡ ENHANCED DEEP OCR SYSTEM")
    print("=" * 50)

    while True:
        print("\nChoose option:")
        print("1. Train enhanced model with improvements")
        print("2. Load and test existing model")
        print("3. Detailed evaluation of existing model")
        print("4. Exit")

        choice = input("Choice: ")

        if choice == "1":
            print("\n🚀 Starting enhanced training...")
            accuracy = train_enhanced_deep_ocr()
            if accuracy is not None:
                print(f"\n🏆 Final result: {accuracy:.4f} word accuracy")
                print("✅ Model saved as 'enhanced_deep_ocr_model.pth'")
            else:
                print("\n❌ Training failed. Check error messages above.")

        elif choice == "2":
            model_path = input("Enter model path (default: enhanced_deep_ocr_model.pth): ")
            if not model_path:
                model_path = "enhanced_deep_ocr_model.pth"

            if not os.path.exists(model_path):
                print(f"❌ Model file not found: {model_path}")
                continue

            try:
                predict_fn = load_and_test_model(model_path)

                while True:
                    image_path = input("\nEnter image path (or 'back' to return): ")
                    if image_path.lower() == 'back':
                        break

                    if not os.path.exists(image_path):
                        print(f"❌ Image not found: {image_path}")
                        continue

                    result = predict_fn(image_path)
                    print(f"🎯 Predicted text: '{result}'")

            except Exception as e:
                print(f"❌ Error loading model: {e}")

        elif choice == "3":
            model_path = input("Enter model path (default: enhanced_deep_ocr_model.pth): ")
            if not model_path:
                model_path = "enhanced_deep_ocr_model.pth"

            if not os.path.exists(model_path):
                print(f"❌ Model file not found: {model_path}")
                continue

            # Load test dataset for evaluation
            dataset_root = "/content/iiit5k_dataset"
            try:
                test_dataset = EnhancedCustomSplitIIIT5KDataset(
                    dataset_root, 'test', 32, 128, 4500, 500, augment=False
                )
                test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                                       collate_fn=ctc_collate_fn, num_workers=2)

                # Load model
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                checkpoint = torch.load(model_path, map_location=device)
                model = EnhancedDeepCRNN(
                    img_height=32,
                    num_classes=checkpoint['num_classes'],
                    hidden_size=checkpoint.get('hidden_size', 256)
                ).to(device)
                model.load_state_dict(checkpoint['model_state_dict'])

                # Run detailed evaluation
                detailed_evaluation(model, test_loader, checkpoint['idx_to_char'], device)

            except Exception as e:
                print(f"❌ Error during evaluation: {e}")

        elif choice == "4":
            print("Goodbye! 👋")
            break

        else:
            print("❌ Invalid choice. Please try again.")

⚡ ENHANCED DEEP OCR SYSTEM

Choose option:
1. Train enhanced model with improvements
2. Load and test existing model
3. Detailed evaluation of existing model
4. Exit
Choice: 2
Enter model path (default: enhanced_deep_ocr_model.pth): /content/deep_ocr_custom_split_model.pth
🔍 Loading trained model...
❌ Error loading model: Error(s) in loading state_dict for EnhancedDeepCRNN:
	Missing key(s) in state_dict: "cnn.3.weight", "cnn.3.bias", "cnn.4.running_mean", "cnn.4.running_var", "cnn.17.weight", "cnn.17.bias", "cnn.17.running_mean", "cnn.17.running_var", "cnn.24.weight", "cnn.24.bias", "cnn.25.weight", "cnn.25.bias", "cnn.25.running_mean", "cnn.25.running_var", "cnn.27.weight", "cnn.27.bias", "cnn.28.weight", "cnn.28.bias", "cnn.28.running_mean", "cnn.28.running_var", "cnn.32.weight", "cnn.32.bias", "cnn.33.weight", "cnn.33.bias", "cnn.33.running_mean", "cnn.33.running_var", "rnn.weight_ih_l2", "rnn.weight_hh_l2", "rnn.bias_ih_l2", "rnn.bias_hh_l2", "rnn.weight_ih_l2_reverse", "rnn.weight

In [ ]:
# Test  the model

import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

# ======================
# 1. DEEP CRNN MODEL DEFINITION (MUST BE INCLUDED!)
# ======================
class DeepCRNN(nn.Module):
    def __init__(self, img_height=32, num_classes=96, hidden_size=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
        )
        self.rnn = nn.LSTM(
            input_size=512,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=False,
            dropout=0.3
        )
        self.classifier = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(0.4)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        conv = self.cnn(x)  # [B, 512, 1, W]
        b, c, h, w = conv.size()
        conv = conv.view(b, c * h, w).permute(2, 0, 1)  # [W, B, 512]
        rnn_out, _ = self.rnn(conv)
        rnn_out = self.dropout(rnn_out)
        output = self.classifier(rnn_out)
        return F.log_softmax(output, dim=2)

# ======================
# 2. CTC DECODE FUNCTION
# ======================
def ctc_decode(log_probs, idx_to_char, blank_idx=0):
    """Greedy CTC decoding"""
    pred_indices = log_probs.argmax(dim=2).squeeze(1).cpu().numpy()  # [T]
    decoded = []
    prev_idx = blank_idx
    for idx in pred_indices:
        if idx != blank_idx and idx != prev_idx:
            if idx in idx_to_char and idx_to_char[idx] != '<BLANK>':
                decoded.append(idx_to_char[idx])
        prev_idx = idx
    return ''.join(decoded)

# ======================
# 3. INFERENCE SETUP
# ======================
model_path = "/content/deep_ocr_custom_split_model.pth"
image_path = "/content/OCR_Hamed5.jpg"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image transform
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((32, 128), antialias=True),
    T.Grayscale(),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

# ======================
# 4. LOAD MODEL
# ======================
print("🚀 Loading trained model...")
checkpoint = torch.load(model_path, map_location=device)
num_classes = checkpoint['num_classes']
idx_to_char = checkpoint['idx_to_char']

# Now we can create the model
model = DeepCRNN(img_height=32, num_classes=num_classes, hidden_size=128).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Model loaded successfully with {num_classes} classes.")

# ======================
# 5. LOAD AND PREPROCESS IMAGE
# ======================
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Cannot load image at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # To 3-channel for ToPILImage
    return transform(img).unsqueeze(0)  # Add batch dim

print(f"🖼️  Preprocessing image: {image_path}")
image_tensor = preprocess_image(image_path).to(device)

# ======================
# 6. RUN INFERENCE
# ======================
print("🔍 Running inference...")
with torch.no_grad():
    log_probs = model(image_tensor)  # [T, 1, num_classes]
    predicted_text = ctc_decode(log_probs, idx_to_char)

# ======================
# 7. OUTPUT RESULT
# ======================
print("\n" + "="*50)
print("🎯 INFERENCE RESULT")
print("="*50)
print(f"Image: {image_path}")
print(f"Predicted Text: '{predicted_text}'")
print("="*50)

🚀 Loading trained model...
✅ Model loaded successfully with 95 classes.
🖼️  Preprocessing image: /content/OCR_Hamed5.jpg
🔍 Running inference...

🎯 INFERENCE RESULT
Image: /content/OCR_Hamed5.jpg
Predicted Text: 'HRERRO'
